In [2]:
import asyncio
from playwright.async_api import async_playwright
import pandas as pd

async def scrape_thirdwaveneame_playwright():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)  # See what's happening
        page = await browser.new_page()
        
        await page.goto("https://thirdwavenearme.com/")
        await page.wait_for_timeout(8000)  # wait for page + cafes to load fully
        
        # Now extract the cafes that are rendered into the page
        cafes = await page.evaluate("""
            () => {
                const elements = Array.from(document.querySelectorAll('div.MuiCardContent-root'));
                const cafes = elements.map(el => {
                    const name = el.querySelector('h2') ? el.querySelector('h2').innerText.trim() : null;
                    const city = el.querySelector('p') ? el.querySelector('p').innerText.trim() : null;
                    return { name, city };
                });
                return cafes.filter(cafe => cafe.name && cafe.city);
            }
        """)
        
        await browser.close()
        
        return pd.DataFrame(cafes)

# HOW TO RUN
df_thirdwave = await scrape_thirdwaveneame_playwright()

# View sample
print(df_thirdwave.head())

# Save it
df_thirdwave.to_csv("raw_thirdwavenearme.csv", index=False)


Empty DataFrame
Columns: []
Index: []
